Due to the handicap that we can only use task's training data and public pre-trained word embeddings, this mostly limits us to pre-transformers era NLP.

One of last major similar competitions was 2018 [jigsaw toxicity competition](https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/) on kaggle. Some ideas from winners:
* [1st place](https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/discussion/52557):
  * diverse pre-trained embeddings
    * "Given that >90% of a model’s complexity resides in the embedding layer, we decided to focus on the embedding layer rather than the post-embedding layers"
    * "Since most of the model complexity lay in the pre-trained embeddings, minor architecture changes made very little impact on score. Additional dense layers, gaussian vs. spatial dropout, additional dropout layers at the dense level, attention instead of max pooling, time distributed dense layers, and more barely changed the overall score of the model."
    * using FastText and Glove embeddings
  * model: "our work-horse was two BiGRU layers feeding into two final Dense layers"
   * translations augmentation - not applicable
   * pseudo-labelling, extensive work on cv+stacking - probably only marginally useful here, not worth the effort
* [2nd place](https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/discussion/52612)
  * ensemble of RNN, DPCNN and GBM models
  * pre-trained embeddings: FastText, Glove twitter, BPEmb, Word2Vec, LexVec
  * translations augmentation - not applicable
* [3rd place](https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/discussion/52762)
  * kitchen sink ensemble of various models from team members
  * fasttest and glove twitter vectors; alone and concatenated
  * models: GRU, LSTM, GRU+CNN, BiLSTM+GRU
  * some char models in the mix instead of usual word/token-based models
* 5th place
  * concatenated glove, fasttext embeddings with subword information
  * main model: 2-level bidirectional gru followed by max pooling and 2 fully-connected layers
  * char-level DPCNN and RNN trained over wordparts
  * 2 layers of stacking

Proposal:
  * bidirectional 2-layer LSTM model with fully connected layers at the head
  * fasttext and glove embeddings (TODO: others), concatenate multiple embeddings together
  * enhance hidden state output (at last timestamp) with its avg/maxpool over time
  * ensemble models across different CV folds and seeds, TODO: checkpoint ensembling, ensemble different archs
  * minor: 0-pad tokens at the beginning so that last hidden state vector is closer to actual tokens; lower learning rate for embedding layer; TODO: cyclic learning rate scheduler

In [8]:
import json
import os
import pickle
import numpy as np
import pandas as pd
import sklearn.model_selection

### Split training data

In [2]:
CV_FOLDS = 10
CV_SEED = 42
CV_PATH_FMT = 'cache/intent/cv{fold}/{split}.json'

In [3]:
train_df = pd.read_json('data/intent/train.json')
eval_df = pd.read_json('data/intent/eval.json')
test_df = pd.read_json('data/intent/test.json')

#TODO: each class should be equally distributed in train and eval fold
#Use sklearn.model_selection.StratifiedKFold instead
cv = sklearn.model_selection.KFold(
    n_splits=CV_FOLDS, shuffle=True, random_state=CV_SEED
    )

for fold_idx, (train_idx, eval_idx) in enumerate(cv.split(train_df.index)):
    for split in ['train', 'eval']:
        filename = CV_PATH_FMT.format(fold=fold_idx, split=split)
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        df = train_df.iloc[train_idx if split == 'train' else eval_idx]
        df.to_json(filename, orient='records')

In [4]:
intents = sorted(set(train_df.intent))
intent2idx = {s: i for (i, s) in enumerate(intents)}
!mkdir -p cache/intent
with open('cache/intent/intent2idx.json', 'w') as fp:
    json.dump(intent2idx, fp, indent=2)

c = train_df.intent.value_counts()
print(f'{len(intents)} classes, training samples per class: {c.min()}..{c.max()}')

150 classes, training samples per class: 100..100


Training data is perfectly balanced!

### Download and parse pre-trained embeddings

In [5]:
# Fasttext: https://fasttext.cc/docs/en/english-vectors.html
!cd cache/ && wget -nc https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip  # 1.5G
!cd cache/ && unzip -n crawl-300d-2M.vec.zip

# Glove: https://nlp.stanford.edu/projects/glove/
!cd cache/ && wget -nc https://nlp.stanford.edu/data/glove.840B.300d.zip  # 2.0G
!cd cache/ && unzip -n glove.840B.300d.zip

# More ideas
# https://separius.github.io/awesome-sentence-embedding/#word-embeddings esp. lexvec, bpemb

File ‘crawl-300d-2M.vec.zip’ already there; not retrieving.

Archive:  crawl-300d-2M.vec.zip
File ‘glove.840B.300d.zip’ already there; not retrieving.

Archive:  glove.840B.300d.zip


In [6]:
# parse .txt with fasttext/glove embeddings
def parse_embedding_txt(path):
    vectors = {}
    dim = 0
    with open(path) as fp:
        for line in fp:
            line = line.split()
            if len(line) == 2: continue  # fasttext header
            if dim == 0:
                dim = len(line) - 1
            elif dim != len(line) - 1:
                continue
            vectors[line[0]] = np.array(line[1:], dtype=np.float32)  # will parse strings
    print('Parsed %d x %dd vectors from %s' % (len(vectors), dim, path))
    return vectors

fasttext_vec = parse_embedding_txt('cache/crawl-300d-2M.vec')
glove_vec = parse_embedding_txt('cache/glove.840B.300d.txt')

Parsed 1999995 x 300d vectors from cache/crawl-300d-2M.vec
Parsed 2195875 x 300d vectors from cache/glove.840B.300d.txt


In [7]:
glove_vec["'t"] = glove_vec["n't"] 
# alias 't to n't for glove
# a quick fix for one fasttext/glove discrepancy

### Generate vocab and embedding matrix

In [8]:
import torch

from dataset import basic_tokenizer
from utils import Vocab

/home/guest/r11922a05/anaconda3/envs/adl-hw1/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# for speed, only leave tokens we would ever encounter
# alt: use full fasttext+glove vocab; +oov tokens? 
# use subword vectors for oov?
vocab = set()
lens = []
for text in list(train_df.text) + list(eval_df.text) + list(test_df.text):
    tokens = basic_tokenizer(text)
    lens += [len(tokens)]
    vocab |= set(tokens)
vocab = Vocab(list(sorted(vocab)))

with open('cache/intent/vocab.pkl', 'wb') as fp:
    pickle.dump(vocab, fp)
with open('cache/intent/vocab.json', 'w') as fp:
    json.dump(vocab.tokens, fp, indent=2)

print(f'vocab size {len(vocab.tokens)}, max len {max(lens)}')

vocab size 6320, max len 29


Concatenate both pre-trained embeddings for 600 dims in total, trim to our vocab, initializing OOV tokens randomly

In [10]:
emb = np.random.normal(
    size=(len(vocab.tokens), 600), loc=0.0, scale=0.2
    )
for token in vocab.tokens:
    i = vocab.token_to_id(token)
    if token in fasttext_vec:
        emb[i, :300] = fasttext_vec[token]
    if token in glove_vec:
        emb[i, 300:] = glove_vec[token]

emb[0, :] = 0.  # zero init the padding token

emb = torch.tensor(emb, dtype=torch.float32)
torch.save(emb, 'cache/intent/embeddings.pt')
print(emb.shape)

torch.Size([6320, 600])


## Cross-validation training 

In [13]:
#-e flag instructs the script to exit on error.
%%bash -e
for f in $(seq 0 9); do
    python3 train_intent.py --data_dir cache/cv$f --ckpt_dir cache/cv$f --device cuda
done

{'epoch': 94, 'lr': 1.0000000000000004e-08, 'train_loss': 0.00032012424656746797, 'val_loss': 0.3760012537240982, 'val_accuracy': 0.95}
{'epoch': 95, 'lr': 1.0000000000000004e-08, 'train_loss': 0.00041850978657680143, 'val_loss': 0.4122757017612457, 'val_accuracy': 0.95}
{'epoch': 96, 'lr': 1.0000000000000004e-08, 'train_loss': 0.0003818906687503738, 'val_loss': 0.37352454662323, 'val_accuracy': 0.95}
{'epoch': 1, 'lr': 0.001, 'train_loss': 2.4717784653294763, 'val_loss': 0.8200955092906952, 'val_accuracy': 0.7586666666666667, 'ckpt': 'cache/cv0/acc0.758667_ep1_run1665369424.ckpt'}
{'epoch': 97, 'lr': 1.0000000000000004e-08, 'train_loss': 0.00038668145319910383, 'val_loss': 0.35815128684043884, 'val_accuracy': 0.95}
{'epoch': 98, 'lr': 1.0000000000000004e-08, 'train_loss': 0.0003802259444060872, 'val_loss': 0.3803877830505371, 'val_accuracy': 0.95}
{'epoch': 2, 'lr': 0.001, 'train_loss': 0.5341311832362751, 'val_loss': 0.40372417867183685, 'val_accuracy': 0.8826666666666667, 'ckpt': 'c

Epoch: 100%|██████████| 100/100 [04:50<00:00,  2.91s/it]


{'epoch': 4, 'lr': 0.001, 'train_loss': 0.17033347554223718, 'val_loss': 0.30458806455135345, 'val_accuracy': 0.9206666666666666, 'ckpt': 'cache/cv0/acc0.920667_ep4_run1665369424.ckpt'}
{'epoch': 5, 'lr': 0.001, 'train_loss': 0.11572333685350868, 'val_loss': 0.2765892744064331, 'val_accuracy': 0.9286666666666666, 'ckpt': 'cache/cv0/acc0.928667_ep5_run1665369424.ckpt'}
{'epoch': 6, 'lr': 0.001, 'train_loss': 0.06716857329538127, 'val_loss': 0.3066303879022598, 'val_accuracy': 0.9266666666666666}
{'epoch': 1, 'lr': 0.001, 'train_loss': 2.569786883187744, 'val_loss': 0.8658385872840881, 'val_accuracy': 0.7273333333333334, 'ckpt': 'cache/cv1/acc0.727333_ep1_run1665369453.ckpt'}
{'epoch': 7, 'lr': 0.001, 'train_loss': 0.05486848665717638, 'val_loss': 0.3641593009233475, 'val_accuracy': 0.9213333333333333}
{'epoch': 2, 'lr': 0.001, 'train_loss': 0.5688153820217781, 'val_loss': 0.5020598471164703, 'val_accuracy': 0.8446666666666667, 'ckpt': 'cache/cv1/acc0.844667_ep2_run1665369453.ckpt'}
{'ep

Epoch: 100%|██████████| 100/100 [09:16<00:00,  5.57s/it]


{'epoch': 94, 'lr': 1.0000000000000004e-08, 'train_loss': 0.00025088017458263083, 'val_loss': 0.2625962719321251, 'val_accuracy': 0.956}
{'epoch': 95, 'lr': 1.0000000000000004e-08, 'train_loss': 0.0002477964199502856, 'val_loss': 0.2837218642234802, 'val_accuracy': 0.956}
{'epoch': 96, 'lr': 1.0000000000000004e-08, 'train_loss': 0.00023506162265993276, 'val_loss': 0.29889705777168274, 'val_accuracy': 0.956}
{'epoch': 1, 'lr': 0.001, 'train_loss': 2.476642757091882, 'val_loss': 0.7813911139965057, 'val_accuracy': 0.76, 'ckpt': 'cache/cv1/acc0.760000_ep1_run1665369987.ckpt'}
{'epoch': 97, 'lr': 1.0000000000000004e-08, 'train_loss': 0.0002798315950404119, 'val_loss': 0.2835979610681534, 'val_accuracy': 0.9553333333333334}
{'epoch': 2, 'lr': 0.001, 'train_loss': 0.5188288722398147, 'val_loss': 0.4808162748813629, 'val_accuracy': 0.8493333333333334, 'ckpt': 'cache/cv1/acc0.849333_ep2_run1665369987.ckpt'}
{'epoch': 98, 'lr': 1.0000000000000004e-08, 'train_loss': 0.00025010300466745766, 'val_

...

{'epoch': 4, 'lr': 0.001, 'train_loss': 0.1580019136715048, 'val_loss': 0.26457083225250244, 'val_accuracy': 0.932, 'ckpt': 'cache/cv1/acc0.932000_ep4_run1665369987.ckpt'}
{'epoch': 5, 'lr': 0.001, 'train_loss': 0.10422710523466175, 'val_loss': 0.30942995846271515, 'val_accuracy': 0.916}
{'epoch': 6, 'lr': 0.001, 'train_loss': 0.08073157000021552, 'val_loss': 0.3109443336725235, 'val_accuracy': 0.93}
{'epoch': 1, 'lr': 0.001, 'train_loss': 2.4900494431549647, 'val_loss': 0.7669132351875305, 'val_accuracy': 0.7833333333333333, 'ckpt': 'cache/cv2/acc0.783333_ep1_run1665370018.ckpt'}
{'epoch': 7, 'lr': 0.001, 'train_loss': 0.05698677078592328, 'val_loss': 0.2266835942864418, 'val_accuracy': 0.9453333333333334, 'ckpt': 'cache/cv1/acc0.945333_ep7_run1665369987.ckpt'}
{'epoch': 8, 'lr': 0.001, 'train_loss': 0.05218422682165116, 'val_loss': 0.29984311759471893, 'val_accuracy': 0.9346666666666666}
{'epoch': 2, 'lr': 0.001, 'train_loss': 0.5478443894183861, 'val_loss': 0.35328468680381775, 'val

Epoch: 100%|██████████| 100/100 [09:17<00:00,  5.57s/it]


{'epoch': 93, 'lr': 1.0000000000000004e-08, 'train_loss': 0.00030834805772240044, 'val_loss': 0.26281774789094925, 'val_accuracy': 0.9573333333333334}
{'epoch': 94, 'lr': 1.0000000000000004e-08, 'train_loss': 0.00029107734848847304, 'val_loss': 0.25866076350212097, 'val_accuracy': 0.956}
{'epoch': 95, 'lr': 1.0000000000000004e-08, 'train_loss': 0.00026983716287639183, 'val_loss': 0.24708804488182068, 'val_accuracy': 0.958}
{'epoch': 96, 'lr': 1.0000000000000004e-08, 'train_loss': 0.000317516071553878, 'val_loss': 0.25811904668807983, 'val_accuracy': 0.9573333333333334}
{'epoch': 1, 'lr': 0.001, 'train_loss': 2.5330159815977202, 'val_loss': 0.898188591003418, 'val_accuracy': 0.7386666666666667, 'ckpt': 'cache/cv2/acc0.738667_ep1_run1665370551.ckpt'}
{'epoch': 97, 'lr': 1.0000000000000004e-08, 'train_loss': 0.00028630284622574074, 'val_loss': 0.2287093847990036, 'val_accuracy': 0.958}
{'epoch': 2, 'lr': 0.001, 'train_loss': 0.5476043332579001, 'val_loss': 0.4258486032485962, 'val_accurac

Epoch: 100%|██████████| 100/100 [09:18<00:00,  5.59s/it]


{'epoch': 5, 'lr': 0.001, 'train_loss': 0.1065837175075738, 'val_loss': 0.2896265536546707, 'val_accuracy': 0.922}
{'epoch': 6, 'lr': 0.001, 'train_loss': 0.07744147739250143, 'val_loss': 0.2608777582645416, 'val_accuracy': 0.9246666666666666, 'ckpt': 'cache/cv2/acc0.924667_ep6_run1665370551.ckpt'}
{'epoch': 1, 'lr': 0.001, 'train_loss': 2.4399929884469733, 'val_loss': 0.8222661018371582, 'val_accuracy': 0.7646666666666667, 'ckpt': 'cache/cv3/acc0.764667_ep1_run1665370582.ckpt'}
{'epoch': 7, 'lr': 0.001, 'train_loss': 0.0679290655315063, 'val_loss': 0.25265611708164215, 'val_accuracy': 0.94, 'ckpt': 'cache/cv2/acc0.940000_ep7_run1665370551.ckpt'}
{'epoch': 8, 'lr': 0.001, 'train_loss': 0.05116861424725151, 'val_loss': 0.2242104783654213, 'val_accuracy': 0.9333333333333333}
{'epoch': 2, 'lr': 0.001, 'train_loss': 0.5228934334415309, 'val_loss': 0.4163997769355774, 'val_accuracy': 0.878, 'ckpt': 'cache/cv3/acc0.878000_ep2_run1665370582.ckpt'}
{'epoch': 9, 'lr': 0.001, 'train_loss': 0.039

In [1]:
%%bash
for x in cache/intent/cv?/best*; do ls -l "$x"; 
python3 test_intent.py --test_file data/intent/eval.json --ckpt_path $x; done

lrwxrwxrwx 1 r11922a05 R11 35 Oct 10 10:40 cache/cv0/best.ckpt -> acc0.949333_ep33_run1665369424.ckpt
Accuracy 0.932000 (2796/3000)
lrwxrwxrwx 1 r11922a05 R11 35 Oct 10 10:49 cache/cv1/best.ckpt -> acc0.956000_ep36_run1665369987.ckpt
Accuracy 0.935000 (2805/3000)
lrwxrwxrwx 1 r11922a05 R11 35 Oct 10 11:00 cache/cv2/best.ckpt -> acc0.952000_ep45_run1665370551.ckpt
Accuracy 0.926000 (2778/3000)
lrwxrwxrwx 1 r11922a05 R11 35 Oct 10 11:07 cache/cv3/best.ckpt -> acc0.946667_ep24_run1665371117.ckpt
Accuracy 0.936333 (2809/3000)
lrwxrwxrwx 1 r11922a05 R11 35 Oct 10 11:07 cache/cv4/best.ckpt -> acc0.952000_ep12_run1665371148.ckpt
Accuracy 0.927333 (2782/3000)
lrwxrwxrwx 1 r11922a05 R11 35 Oct 10 01:28 cache/cv5/best.ckpt -> acc0.949333_ep36_run1665336378.ckpt
Accuracy 0.934667 (2804/3000)
lrwxrwxrwx 1 r11922a05 R11 35 Oct 10 01:32 cache/cv6/best.ckpt -> acc0.964667_ep35_run1665336668.ckpt
Accuracy 0.933333 (2800/3000)
lrwxrwxrwx 1 r11922a05 R11 35 Oct 10 01:37 cache/cv7/best.ckpt -> acc0.96333

# Training with whole data

In [9]:
%%bash -e
python3 train_intent.py --data_dir data/intent --ckpt_dir ckpt/intent --device cuda

Error while terminating subprocess (pid=7189): 


# Testing


In [ ]:
%%bash -e
python3 test_intent.py --test_file data/intent/test.json --ckpt_path ckpt/intent/best.ckpt; done